In [2]:
!pip install srtm.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('/kaggle/input/no2-estimation/Train.csv')
test = pd.read_csv('/kaggle/input/no2-estimation/Test.csv')
ss = pd.read_csv('/kaggle/input/no2-estimation/SampleSubmission.csv')
train.head()

,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure,GT_NO2
0,ID_ENTGC7,1/1/19,PD01,45.601585,11.903551,0.000000,NaN,0.230527,0.559117,0.000024,0.000117,NaN,14440.82126,31.0
1,ID_8JCCXC,1/1/19,PD04,45.371005,11.840830,3.047342,NaN,-0.074006,0.869309,0.000024,0.000127,NaN,14441.79815,42.0
2,ID_V3136Z,1/1/19,RO01,45.045825,12.060869,0.000000,NaN,0.024470,0.674160,0.000024,0.000086,NaN,14437.38294,31.0
3,ID_KRVZDJ,1/1/19,RO02,45.104075,11.553241,1.200467,NaN,-0.010442,0.920054,0.000024,0.000124,NaN,14440.83831,30.0
4,ID_PR351A,1/1/19,RO03,45.038758,11.790152,1.274564,NaN,-0.176178,0.747464,0.000024,0.000116,NaN,14438.79037,58.0


In [16]:
test['id'] = test.index

In [4]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day
train['WeekOfYear'] = train['Date'].dt.isocalendar().week

test['Year'] = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['WeekOfYear'] = test['Date'].dt.isocalendar().week

/tmp/ipykernel_30/3517945967.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Date'] = pd.to_datetime(train['Date'])
/tmp/ipykernel_30/3517945967.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test['Date'] = pd.to_datetime(test['Date'])


In [5]:
import pandas as pd
import srtm
from geopy.geocoders import Nominatim

# Initialize SRTM elevation data
elevation_data = srtm.get_data()



# Function to get elevation for a given latitude and longitude
def get_elevation(lat, lon):
    return elevation_data.get_elevation(lat, lon)

# Apply the function to the DataFrame to get the elevation column
train['elevation_meters'] = train.apply(lambda row: get_elevation(row['LAT'], row['LON']), axis=1)
test['elevation_meters'] = test.apply(lambda row: get_elevation(row['LAT'], row['LON']), axis=1)

# Display the DataFrame with latitude, longitude, and elevation columns
train.head()


Creating /root/.cache/srtm
4 2884802
4 2884802
4 2884802
4 2884802
4 2884802
4 2884802
4 2884802


,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure,GT_NO2,Year,Month,Day,WeekOfYear,elevation_meters
0,ID_ENTGC7,2019-01-01,PD01,45.601585,11.903551,0.000000,NaN,0.230527,0.559117,0.000024,0.000117,NaN,14440.82126,31.0,2019,1,1,1,25
1,ID_8JCCXC,2019-01-01,PD04,45.371005,11.840830,3.047342,NaN,-0.074006,0.869309,0.000024,0.000127,NaN,14441.79815,42.0,2019,1,1,1,10
2,ID_V3136Z,2019-01-01,RO01,45.045825,12.060869,0.000000,NaN,0.024470,0.674160,0.000024,0.000086,NaN,14437.38294,31.0,2019,1,1,1,-4
3,ID_KRVZDJ,2019-01-01,RO02,45.104075,11.553241,1.200467,NaN,-0.010442,0.920054,0.000024,0.000124,NaN,14440.83831,30.0,2019,1,1,1,7
4,ID_PR351A,2019-01-01,RO03,45.038758,11.790152,1.274564,NaN,-0.176178,0.747464,0.000024,0.000116,NaN,14438.79037,58.0,2019,1,1,1,2


In [6]:
train["NO2_Ratio"] = train["NO2_trop"] / train["NO2_strat"]
train["SUM"] = train["NO2_strat"] + train["NO2_total"] + train["NO2_trop"]
train["LSTsqr"] = train["LST"] ** 2
train["Precipitationsqr"] = train["Precipitation"] ** 2

In [7]:
test["NO2_Ratio"] = test["NO2_trop"] / test["NO2_strat"]
test["SUM"] = test["NO2_strat"] + test["NO2_total"] + test["NO2_trop"]
test["LSTsqr"] = test["LST"] ** 2
test["Precipitationsqr"] = test["Precipitation"] ** 2

In [8]:
test['elevation_meters'].unique()

array([ 11,   7, 204,  40,  76,  23])

In [9]:
need_col = ['Precipitation', 'LST', 'AAI',
       'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop',
       'TropopausePressure', 'elevation_meters', 'Month','WeekOfYear','NO2_Ratio', 'SUM', 'LSTsqr', 'Precipitationsqr']

In [17]:
train_11 = train[train.elevation_meters==11]
train_11 = train_11.reset_index(drop=True)
train_11.head()

,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,...,GT_NO2,Year,Month,Day,WeekOfYear,elevation_meters,NO2_Ratio,SUM,LSTsqr,Precipitationsqr
0,ID_OB4V7B,2019-01-01,X10167,45.016878,11.076095,2.950671,NaN,0.121945,0.955316,0.000024,...,31.275,2019,1,1,1,11,NaN,NaN,NaN,8.706461
1,ID_XJ1OU9,2019-02-01,X10167,45.016878,11.076095,0.000000,NaN,-0.436368,0.620831,0.000035,...,24.550,2019,2,1,5,11,NaN,NaN,NaN,0.000000
2,ID_11XZIC,2019-03-01,X10167,45.016878,11.076095,0.000000,NaN,-0.651168,0.109905,0.000030,...,31.700,2019,3,1,9,11,NaN,NaN,NaN,0.000000
3,ID_4JA7ZZ,2019-04-01,X10167,45.016878,11.076095,0.000000,NaN,-1.402872,0.146766,0.000022,...,36.675,2019,4,1,14,11,NaN,NaN,NaN,0.000000
4,ID_OIFDH1,2019-05-01,X10167,45.016878,11.076095,0.000000,NaN,-1.042541,0.435450,0.000017,...,46.125,2019,5,1,18,11,NaN,NaN,NaN,0.000000


In [12]:
print(train_11.shape)

(1096, 23)


In [14]:
test['id'] = test.index

In [15]:
test_sub = pd.merge(ss, test[['id', 'ID_Zindi','elevation_meters']], how='left', on='ID_Zindi')
test_sub

,ID_Zindi,GT_NO2,id,elevation_meters
0,ID_003WOR,0,4050,11
1,ID_009Y2B,0,1942,76
2,ID_01C05M,0,4282,76
3,ID_0216G3,0,1143,40
4,ID_023C6G,0,4044,11
...,...,...,...,...
6571,ID_ZVEQ52,0,824,204
6572,ID_ZWZ1IA,0,4948,76
6573,ID_ZX7B4A,0,4043,23
6574,ID_ZYZLPS,0,4051,7


In [13]:
lgbm_1000 = pd.read_csv('/kaggle/input/sub-lgbm/sub_lgbm_1000.csv')
lgbm_1000.head()

,ID_Zindi,GT_NO2
0,ID_003WOR,15.006042
1,ID_009Y2B,25.157996
2,ID_01C05M,33.077226
3,ID_0216G3,15.821761
4,ID_023C6G,18.380325


In [18]:
test_11 = test[test.elevation_meters==11]
test_11 = test_11.reset_index(drop=True)
print(test_11.shape)
test_11.head()

(1096, 23)


,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,...,Year,Month,Day,WeekOfYear,elevation_meters,NO2_Ratio,SUM,LSTsqr,Precipitationsqr,id
0,ID_2MYNQS,2019-01-01,PD03,45.289376,11.642394,3.277529,NaN,-0.313361,0.771456,0.000024,...,2019,1,1,1,11,NaN,NaN,NaN,10.742196,0
1,ID_Z9ICW9,2019-01-02,PD03,45.289376,11.642394,0.000000,281.08,-0.177263,0.166926,0.000033,...,2019,1,2,1,11,1.718182,0.000218,79005.9664,0.000000,6
2,ID_AWH9CU,2019-01-03,PD03,45.289376,11.642394,0.000000,274.18,-0.716974,0.000000,0.000029,...,2019,1,3,1,11,1.252595,0.000130,75174.6724,0.000000,12
3,ID_U7XGZ3,2019-01-04,PD03,45.289376,11.642394,3.277529,277.98,-1.160514,0.000000,0.000021,...,2019,1,4,1,11,3.644550,0.000196,77272.8804,10.742196,18
4,ID_T2H0JJ,2019-01-05,PD03,45.289376,11.642394,0.000000,NaN,-0.887354,0.543159,0.000017,...,2019,1,5,1,11,NaN,NaN,NaN,0.000000,24


In [19]:
need_col = ['Precipitation', 'LST', 'AAI',
       'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop',
       'TropopausePressure', 'Month','WeekOfYear','NO2_Ratio', 'SUM', 'LSTsqr', 'Precipitationsqr']

In [20]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
test_preds=[]
rmse=[]

# Initialize KFold for 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# LightGBM parameters (you can adjust these)
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'n_estimators': 900,
    'early_stopping_rounds': 100,
    'verbose':-1 
}



# List to store validation data with IDs, predictions, and true values
validation_results = []

# Iterate through each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(train_11, train_11['GT_NO2'])):
    print(f"Training fold {fold + 1}...")

    X_train, X_val = train_11.iloc[train_idx], train_11.iloc[val_idx]
    y_train, y_val = train_11['GT_NO2'].iloc[train_idx], train_11['GT_NO2'].iloc[val_idx]
    # Initialize and train the LightGBM regressor
    lgb_model = lgb.LGBMRegressor(**lgb_params)
    lgb_model.fit(
        X_train[need_col], y_train,
        eval_set=[(X_val[need_col], y_val)],
        eval_metric='rmse',
         # Set to 1 if you want to see training logs
    )
    
    # Predict on validation data and store the out-of-fold predictions
    val_preds = lgb_model.predict(X_val[need_col])
    #oof_preds[val_idx] = val_preds
    
    # Stack the validation results (IDs, predicted values, true values)
    validation_results.append(pd.DataFrame({
        'id': val_idx,
        'true_value': y_val,
        'predicted_value': val_preds
    }))
    
    # Calculate RMSE for the validation data in the current fold
    #fold_rmse = mean_squared_error(y_val, val_preds, squared=False)
    #print(f"Fold {fold + 1} RMSE: {fold_rmse}")
    mask = ~np.isnan(y_val) & ~np.isnan(val_preds)

    # Calculate RMSE only for non-NaN values
    fold_rmse = mean_squared_error(y_val[mask], val_preds[mask], squared=False)
    print(f"Fold {fold + 1} RMSE (ignoring NaN): {fold_rmse}")

    # Predict on test data for the current fold
    pred = lgb_model.predict(test_11[need_col])
    test_preds.append(pred)
    rmse.append(fold_rmse)
# Concatenate all validation results into one DataFrame
validation_df = pd.concat(validation_results, ignore_index=True)

# Calculate the mean of the test predictions from all folds
#final_test_preds = test_preds.mean(axis=1)

# Calculate the overall RMSE for the out-of-fold predictions
#overall_rmse = mean_squared_error(y, oof_preds, squared=False)
print(f"Overall OOF RMSE: {np.mean(rmse)}")

# Display validation DataFrame (ID, True Values, Predicted Values)
#print(validation_df)

# Now you can compare validation_df's 'true_value' with 'predicted_value'


Training fold 1...
Fold 1 RMSE (ignoring NaN): 5.505812549292625
Training fold 2...
Fold 2 RMSE (ignoring NaN): 5.91665660194937
Training fold 3...
Fold 3 RMSE (ignoring NaN): 6.0656651626694025
Training fold 4...
Fold 4 RMSE (ignoring NaN): 6.1556342235714006
Training fold 5...
Fold 5 RMSE (ignoring NaN): 5.773022849469894
Training fold 6...
Fold 6 RMSE (ignoring NaN): 6.574919901927828
Training fold 7...
Fold 7 RMSE (ignoring NaN): 6.173710667839284
Training fold 8...
Fold 8 RMSE (ignoring NaN): 4.771278882870241
Training fold 9...
Fold 9 RMSE (ignoring NaN): 5.298585097619557
Training fold 10...
Fold 10 RMSE (ignoring NaN): 6.565100859654606
Overall OOF RMSE: 5.880038679686421


In [21]:
test_preds_array = np.array(test_preds)  # Convert list of predictions to a NumPy array

# Calculate the mean predictions across the folds
final_test_preds = test_preds_array.mean(axis=0)  # Change axis=1 to axis=0 for correct mean calculation

In [22]:
final_test_preds

array([24.06405128, 15.76578255, 18.26898658, ..., 24.98958286,
       18.62551814, 24.38078227])

In [23]:
test_11['pred'] = final_test_preds

In [25]:
test_sub = pd.merge(lgbm_1000, test_11[['id', 'ID_Zindi','pred','elevation_meters']], how='left', on='ID_Zindi')
test_sub

,ID_Zindi,GT_NO2,id,pred,elevation_meters
0,ID_003WOR,15.006042,4050.0,15.471109,11.0
1,ID_009Y2B,25.157996,NaN,NaN,NaN
2,ID_01C05M,33.077226,NaN,NaN,NaN
3,ID_0216G3,15.821761,NaN,NaN,NaN
4,ID_023C6G,18.380325,4044.0,16.710630,11.0
...,...,...,...,...,...
6571,ID_ZVEQ52,11.994950,NaN,NaN,NaN
6572,ID_ZWZ1IA,10.019211,NaN,NaN,NaN
6573,ID_ZX7B4A,23.383131,NaN,NaN,NaN
6574,ID_ZYZLPS,19.525151,NaN,NaN,NaN


In [ ]:
df['new_col'] = np.where(pd.isna(df['col1']), 'not NaN', df['col2'])

In [29]:
test_sub['pred_new'] = np.where(pd.isna(test_sub['pred']), test_sub['GT_NO2'], test_sub['pred'])

In [30]:
test_sub

,ID_Zindi,GT_NO2,id,pred,elevation_meters,pred_new
0,ID_003WOR,15.006042,4050.0,15.471109,11.0,15.471109
1,ID_009Y2B,25.157996,NaN,NaN,NaN,25.157996
2,ID_01C05M,33.077226,NaN,NaN,NaN,33.077226
3,ID_0216G3,15.821761,NaN,NaN,NaN,15.821761
4,ID_023C6G,18.380325,4044.0,16.710630,11.0,16.710630
...,...,...,...,...,...,...
6571,ID_ZVEQ52,11.994950,NaN,NaN,NaN,11.994950
6572,ID_ZWZ1IA,10.019211,NaN,NaN,NaN,10.019211
6573,ID_ZX7B4A,23.383131,NaN,NaN,NaN,23.383131
6574,ID_ZYZLPS,19.525151,NaN,NaN,NaN,19.525151


In [31]:
test_sub['GT_NO2'] = test_sub['pred_new']

In [32]:
ss_new = test_sub[['ID_Zindi','GT_NO2']]

In [33]:
ss_new.to_csv('lgbm_11.csv',index=False)

In [34]:
ss_new

,ID_Zindi,GT_NO2
0,ID_003WOR,15.471109
1,ID_009Y2B,25.157996
2,ID_01C05M,33.077226
3,ID_0216G3,15.821761
4,ID_023C6G,16.710630
...,...,...
6571,ID_ZVEQ52,11.994950
6572,ID_ZWZ1IA,10.019211
6573,ID_ZX7B4A,23.383131
6574,ID_ZYZLPS,19.525151
